# References

GDS allows defining the component once in memory and reference to that structure in other components.

![](https://i.imgur.com/D4qaJpi.png)

As you build complex circuit you will make circuits that combine reference to other simpler circuits. Adding a reference is like having a pointer to the other devices.

The GDSII specification allows the use of references, and similarly gdsfactory uses them (with the `add_ref()` function).  So what is a reference? Simply put:  **A reference does not contain any geometry. It only *points* to an existing geometry**.  

Say Alice has a ridiculously large polygon with 100 billion vertices in it that we'll call BigPolygon. It's huge, and she needs to use it in her design 250 times.  Well, a single copy of BigPolygon takes up 100GB of memory just by itself, so she doesn't want to make 250 copies of it. Since Alice is clever, she instead *references* the polygon 250 times.  Each reference only uses a few bytes of memory -- it only needs to know the memory address of BigPolygon and a few other things. In this way, she can keep one copy of BigPolygon and use it again and again wherever she needs to.


Let's start by making a blank geometry (`Component`) then adding a single polygon
to it. 

In [ ]:
import numpy as np
import gdsfactory as gf

gf.config.set_plot_options(show_subports=False)


# Create a blank Component
p = gf.Component()

# Add a polygon
xpts = [0,0,5,6,9,12]
ypts = [0,1,1,2,2,0]
p.add_polygon([xpts,ypts])

# Quickplot the Component with the polygon in it
p

Now, pretend we're in the same position as Alice: We want to reuse this polygon
repeatedly but do not want to make multiple copies of it.  To do so, we need to
make a second blank `Component`, this time called `D`.  In this new Component we'll
*reference* our Component `P` which contains our polygon.

In [ ]:
c = gf.Component()             # Create a new blank Component
poly_ref = c.add_ref(p)  # Reference the Component "p" that has the polygon in it
c # Quickplot the reference-containing Component "c"

OK, well that seemed to work, but it also seems thoroughly useless!  It looks
like you just made a copy of your polygon -- but remember, you didn't actually
make a second polygon, you just made a reference (aka pointer) to the original
polygon.  Let's add two more references to `c`:

In [ ]:
poly_ref2 = c.add_ref(p)  # Reference the Component "p" that has the polygon in it
poly_ref3 = c.add_ref(p)  # Reference the Component "p" that has the polygon in it

c # Quickplot the reference-containing Component "c"

Now you have 3x polygons all on top of each other.  Again, this would appear
useless, except that you can manipulate each reference indepedently. Notice that
when you called `c.add_ref(p)` above, we saved the result to a new variable each
time (`poly_ref`, `poly_ref2`, and `poly_ref3`)?  You can use those variables to
reposition the references. 

In [ ]:
poly_ref2.rotate(15) # Rotate the 2nd reference we made 15 degrees
poly_ref3.rotate(30) # Rotate the 3rd reference we made 30 degrees
c

Now you're getting somewhere!  You've only had to make the polygon once, but you're
able to reuse it as many times as you want.

## Modifying the referenced geometry

What happens when you change the original geometry that the reference points to?  In your case, your references in
`c` all point to the Component `p` that with the original polygon.  Let's try
adding a second polygon to `p`.

First you add the second polygon and make sure `P` looks like you expect:

In [ ]:
# Add a 2nd polygon to "p"
xpts = [14,14,16,16]
ypts = [0,2,2,0]
p.add_polygon([xpts,ypts], layer = (1,0))
p

That looks good.  Now let's find out what happened to `c` that contains the
three references.  Keep in mind that you have not modified `c` or executed any
functions/operations on `c` -- all you have done is modify `p`.

In [ ]:
c

 **When you modify the original geometry, all of the
references automatically reflect the modifications.**  This is very powerful,
because you can use this to make very complicated designs from relatively simple
elements in a computation- and memory-efficienct way.

Let's try making references a level deeper by referencing `c`.  Note here we use
the `<<` operator to add the references -- this is just shorthand, and is
exactly equivalent to using `add_ref()`

In [ ]:
c2 = gf.Component()             # Create a new blank Component
d_ref1 = c2.add_ref(c)  # Reference the Component "c" that 3 references in it
d_ref2 = c2 << c        # Use the "<<" operator to create a 2nd reference to c
d_ref3 = c2 << c        # Use the "<<" operator to create a 3rd reference to c

d_ref1.move([20,0])
d_ref2.move([40,0])

c2

As you've seen you have two ways to add a reference to our component:

1. create the reference and add it to the component

In [ ]:
c = gf.Component('reference_sample')
w = gf.components.straight(width=0.6)
wr = w.ref()
c.add(wr)
c

2. or do it in a single line

In [ ]:
c = gf.Component('reference_sample_shorter_syntax')
wr = c << gf.components.straight(width=0.6)
c

in both cases you can move the reference `wr` after created

In [ ]:
import gdsfactory as gf
c = gf.Component('two_references')
wr1 = c << gf.components.straight(width=0.6)
wr2 = c << gf.components.straight(width=0.6)
wr2.movey(10)
c.add_ports(wr1.get_ports_list(), prefix='top_')
c.add_ports(wr2.get_ports_list(), prefix='bot_')
c

In [ ]:
c.ports

You can also auto_rename ports using gdsfactory default convention, where ports are numbered clockwise starting from the bottom left

In [ ]:
c.auto_rename_ports()

In [ ]:
c.ports

In [ ]:
c

## Arrays of references

In GDS, there's a type of structure called a "CellArray" which takes a cell and repeats it NxM times on a fixed grid spacing. For convenience, `Component` includes this functionality with the add_array() function. 
Note that CellArrays are not compatible with ports (since there is no way to access/modify individual elements in a GDS cellarray) 

gdsfactory also provides with more flexible arrangement options if desired, see for example `grid()` and `packer()`.

As well as `gf.components.array` and `gf.components.array_2d`

Let's make a new Component and put a big array of our Component `c` in it:

In [ ]:
c3 = gf.Component() # Create a new blank Component
aref = c3.add_array(c, columns = 6, rows = 3, spacing = [20, 15])  # Reference the Component "c" 3 references in it with a 3 rows, 6 columns array
c3

CellArrays don't have ports and there is no way to access/modify individual elements in a GDS cellarray.

gdsfactory provides you with similar functions in `gf.components.array` and `gf.components.array_2d`

In [ ]:
c4 = gf.Component() # Create a new blank Component
aref = c4<< gf.c.array(component=c, n=3)
c4.add_ports(aref.get_ports_list())
c4

In [ ]:
gf.c.array?

In [ ]:
gf.c.array_2d?

In [ ]:
c5 = gf.Component() 
aref = c5<< gf.c.array_2d(component=c, cols=6, rows=3, pitch_x=20, pitch_y=15)
c5.add_ports(aref.get_ports_list())
c5

You can also create an array of references for periodic structures. Lets create a [Distributed Bragg Reflector](https://picwriter.readthedocs.io/en/latest/components/dbr.html)


In [ ]:
@gf.cell
def dbr_cell(w1=0.5, w2=0.6, l1=0.2, l2=0.4, straight_function=gf.components.straight):
    c = gf.Component()
    c1 = c << straight_function(length=l1, width=w1)
    c2 = c << straight_function(length=l2, width=w2)
    c2.connect(port=1, destination=c1.ports[2])
    c.add_port(1, port=c1.ports[1])
    c.add_port(2, port=c2.ports[2])
    return c


w1 = 0.5
w2 = 0.6
l1 = 0.2
l2 = 0.4
n = 3
straight_function = gf.components.straight
c = gf.Component('reference_array')
cell = dbr_cell(w1=w1, w2=w2, l1=l1, l2=l2, straight_function=straight_function)
cell

In [ ]:
c = gf.Component('DBR')
c.add_array(cell, columns=n, rows=1, spacing=(l1 + l2, 100))
c

Finally we need to add ports to the new component

In [ ]:
p0 = c.add_port(1, port=cell.ports[1])
p1 = c.add_port(2, port=cell.ports[2])
p1.midpoint = [(l1 + l2) * n, 0]
c

## Connect references

We have seen that once you create a reference you can manipulate the reference to move it to a location. Here we are going to connect that reference to a port. Remeber that we follow that a certain reference `source` connects to a `destination` port

In [ ]:
gf.components.bend_circular()

In [ ]:
c = gf.Component("sample_reference_connect")

mmi = c << gf.components.mmi1x2()
b = c << gf.components.bend_circular()

b.connect(1, destination=mmi.ports[2])
c

## Port naming

You have the freedom to name the ports as you want, and you can use `gf.port.auto_rename_ports` to rename them later on.

Here is the default naming convention.


Ports are numbered clock-wise starting from the bottom left corner


In [ ]:
import gdsfactory as gf

size = 4
c = gf.components.nxn(west=2, south=2, north=2, east=2, xsize=size, ysize=size)
c

In [ ]:
c = gf.c.straight_heater_metal()
c

In [ ]:
c.ports

You can get the optical ports by `layer`

In [ ]:
c.get_ports_dict(layer=(1,0))

or by `width`

In [ ]:
c.get_ports_dict(width=0.5)

**Enforcing naming convention**. As you add references to the component you don't have to name the exposed ports correctly,
As long as you name the ports uniquely you can just call `auto_rename_ports()` 

The default conventions are:


- auto_rename_ports: both electrical and optical ports are named clockwise
- auto_rename_ports_layer_orientation: adds prefix layer, as well as direction `N` for north, `S` for south ...


In [ ]:
c = gf.c.straight_heater_metal()
c.auto_rename_ports_layer_orientation()
c.ports

In [ ]:
c.auto_rename_ports()
c.ports

You can even rename them with a different port naming convention

- prefix: add `e` for electrical `o` for optical
- clockwise
- counter-clockwise
- orientation `E` East, `W` West, `N` North, `S` South

In [ ]:
import gdsfactory as gf

c = gf.Component()
nxn =  gf.c.nxn(west=1, north=2, east=3, south=4)
ref = c << nxn
c

In [ ]:
ref.ports_layer

In [ ]:
c = gf.Component()
nxn = gf.c.nxn(west=1, north=2, east=3, south=4)
ref = c.add_ref(nxn)
ref.rotate(-90)
c

In [ ]:
ref.ports_layer

You can still reference the port by their orientation. 

For example, lets extend the top East facing port

In [ ]:
import gdsfactory as gf

nxn = gf.c.nxn(west=1, north=2, east=3, south=4)
c = gf.c.extension.extend_ports(component=nxn, port_names = [nxn.ports_layer['1_0_W0']])
c

## component_sequence

When you have repetitive connections you can describe the connectivity as an ASCII map

In [ ]:
import gdsfactory as gf

In [ ]:
bend180 = gf.components.bend_circular180()
wg_pin = gf.components.straight_pin()
wg = gf.components.straight()

# Define a map between symbols and (component, input port, output port)
symbol_to_component = {
    "D": (bend180, 1, 2),
    "C": (bend180, 2, 1),
    "P": (wg_pin, 1, 2),
    "-": (wg, 1, 2),
}

# Generate a sequence
# This is simply a chain of characters. Each of them represents a component
# with a given input and and a given output

sequence = "DC-P-P-P-P-CD"
component = gf.components.component_sequence(sequence=sequence, symbol_to_component=symbol_to_component)
component.name = 'component_sequence'
component

As the sequence is defined as a string you can use the string operations to build complicated sequences